## 1. Instalação e Setup

In [ ]:
# YOLO + StrongSORT + OpenCV + Supervision
!pip install ultralytics supervision opencv-python --quiet


In [2]:
# Importação das bibliotecas necessárias
from ultralytics import YOLO
import cv2
import os
from pathlib import Path
import matplotlib.pyplot as plt

## 2. Carregando um modelo pré-treinado

In [ ]:
# Carregando o modelo YOLOv8n (nano - versão mais leve)
model = YOLO('yolov8x.pt')

# Também podemos carregar diferentes tamanhos de modelo:
# model = YOLO('yolov8s.pt')  # small
# model = YOLO('yolov8m.pt')  # medium
# model = YOLO('yolov8l.pt')  # large
# model = YOLO('yolov8x.pt')  # extra large

# Ativando o StrongSORT como rastreador
model.trackers = "botsort.yaml"  # pode trocar por 'strongsort.yaml' se preferir

## 3. Função auxiliar de visualização

In [27]:
def mostrar_deteccao(imagem, resultados):
    # Mostra a imagem com as detecções do YOLOv8
    if isinstance(imagem, str):
        imagem = cv2.imread(imagem)
        imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(resultados[0].plot())
    plt.axis('off')
    plt.show()

    # Printando detalhes das detecções
    caixas = resultados[0].boxes
    print(f"\nForam detectados {len(caixas)} objetos na imagem.")

    for i, box in enumerate(caixas):
        classe = int(box.cls[0])
        nome = model.names[classe]
        confianca = float(box.conf[0])
        print(f"Detecção {i+1}: {nome} (confiança: {confianca:.2f})")

## 4. Função para explorar os resultados

In [28]:
def explorar_resultados(resultados):
    if not resultados or not hasattr(resultados[0], "boxes") or len(resultados[0].boxes) == 0:
        print("Nenhum resultado encontrado para explorar.")
        return

    resultado = resultados[0]

    print("\n=== Informações do resultado ===")
    print(f"Tipo do objeto resultado: {type(resultado)}")
    print(f"Quantidade de boxes (detecções): {len(resultado.boxes)}")

    print("\n--- Detalhes de cada box ---")
    for i, box in enumerate(resultado.boxes):
        print(f"Box {i}:")
        print(f"  Coordenadas: {box.xyxy.cpu().numpy()}")
        print(f"  Classe: {box.cls.cpu().numpy()}")
        print(f"  Confiança: {box.conf.cpu().numpy()}")

    print("\nBounding boxes (todas):")
    print(resultado.boxes.xyxy.cpu().numpy())

    print("\nClasses detectadas (índices):")
    print(resultado.boxes.cls.cpu().numpy())

    print("\nNomes das classes detectadas:")
    for c in resultado.boxes.cls:
        print(model.names[int(c)])

    print("\nScores de confiança:")
    print(resultado.boxes.conf.cpu().numpy())
    print("=== Fim dos resultados ===\n")

## 5. Detecção em imagem com retorno e visualização

In [29]:
# Crie o dicionário só uma vez, antes de rodar as detecções
localizacoes_motos = {}

def detectar_moto_na_imagem(caminho):
    print(f"\nProcessando imagem: {caminho}")
    resultados = model.predict(
        source=caminho,
        conf=0.5,
        iou=0.7,
        max_det=20,
        classes=[3],  # Só motos
        save=True,
        project='resultados_imagens',
        name='detectados',
        exist_ok=True
    )
    mostrar_deteccao(caminho, resultados)
    explorar_resultados(resultados)

    # Salva as coordenadas das motos detectadas
    if resultados and hasattr(resultados[0], "boxes") and len(resultados[0].boxes) > 0:
        for box in resultados[0].boxes.xyxy.cpu().numpy():
            novo_id = len(localizacoes_motos)
            localizacoes_motos[novo_id] = box
            print(f"  Moto detectada! ID: {novo_id} | Coordenadas: {box}")
    else:
        print("  Nenhuma moto detectada nesta imagem.")

## 6. Executar detecção em todos arquivos da pasta

In [ ]:
# Extensões de imagem que vamos aceitar
extensoes_imagem = ['.jpg', '.jpeg', '.png']

# Listando arquivos do diretório atual
arquivos = os.listdir('.')

# Filtrando só as imagens
imagens = [f for f in arquivos if Path(f).suffix.lower() in extensoes_imagem]

# Rodando detecção em cada imagem e salvando as localizações
for img in imagens:
    detectar_moto_na_imagem(img)

## 7. registrar localização de motos detectadas em fotos

In [31]:
def mostrar_posicao_moto(id_moto):
    print(f"Buscando posição da moto com ID {id_moto}...")
    if id_moto in localizacoes_motos:
        coord = localizacoes_motos[id_moto]
        print(f"Moto ID {id_moto} está na posição (bounding box): {coord}")
    else:
        print(f"Moto ID {id_moto} não encontrada. Tente outro número!")

## 8. exibição em mapa 2D

In [32]:
def mapa_2d_motos():
    # Mostra um gráfico com as posições das motos detectadas
    if not localizacoes_motos:
        print("Não tem nenhuma moto pra mostrar no mapa.")
        return

    plt.figure(figsize=(10, 6))
    plt.title("Mapa 2D das Motos Detectadas")
    plt.xlabel("Eixo X (largura da imagem)")
    plt.ylabel("Eixo Y (altura da imagem)")
    plt.grid(True)

    for id_moto, (x1, y1, x2, y2) in localizacoes_motos.items():
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        plt.plot(cx, cy, "ro")
        plt.text(cx + 5, cy, f"ID {id_moto}", fontsize=10, color='blue')

    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

In [ ]:
# Exibe todas as motos detectadas no terminal
print("IDs das motos detectadas:", list(localizacoes_motos.keys()))
for id_moto in localizacoes_motos:
    mostrar_posicao_moto(id_moto)

# Exibe o mapa 2D com todas as motos
mapa_2d_motos()

##  9. Lista de classes COCO

In [16]:
# Lista de todas as classes que o modelo reconhece
for idx, class_name in model.names.items():
    print(f"{idx}: {class_name}")


0: person
1: bicycle
2: car
3: motorcycle
4: airplane
5: bus
6: train
7: truck
8: boat
9: traffic light
10: fire hydrant
11: stop sign
12: parking meter
13: bench
14: bird
15: cat
16: dog
17: horse
18: sheep
19: cow
20: elephant
21: bear
22: zebra
23: giraffe
24: backpack
25: umbrella
26: handbag
27: tie
28: suitcase
29: frisbee
30: skis
31: snowboard
32: sports ball
33: kite
34: baseball bat
35: baseball glove
36: skateboard
37: surfboard
38: tennis racket
39: bottle
40: wine glass
41: cup
42: fork
43: knife
44: spoon
45: bowl
46: banana
47: apple
48: sandwich
49: orange
50: broccoli
51: carrot
52: hot dog
53: pizza
54: donut
55: cake
56: chair
57: couch
58: potted plant
59: bed
60: dining table
61: toilet
62: tv
63: laptop
64: mouse
65: remote
66: keyboard
67: cell phone
68: microwave
69: oven
70: toaster
71: sink
72: refrigerator
73: book
74: clock
75: vase
76: scissors
77: teddy bear
78: hair drier
79: toothbrush
